In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd

# Importing data

In [3]:
raw_data = pd.read_csv("../data/train.csv")

In [4]:
clean_data = raw_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)

In [5]:
clean_data = pd.get_dummies(clean_data).fillna(-1)

In [6]:
train, test = train_test_split(clean_data)

In [7]:
def split_x_y(dataframe, target):
    return dataframe.drop(target, axis=1), dataframe[target]

In [8]:
X_train, y_train = split_x_y(train, "Survived")
X_test, y_test = split_x_y(test, "Survived")

# Baseline model

In [9]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [10]:
y_pred = rf.predict_proba(X_test)

In [11]:
auc = roc_auc_score(y_true=y_test, y_score=y_pred[:, 1])

In [12]:
auc

0.8424081632653061

# Deep Forest

## By Hand

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4)
rf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10)

In [15]:
rf1.fit(X_train, y_train)
y_pred_train_1 = rf1.predict_proba(X_train)
y_pred_test_1 = rf1.predict_proba(X_test)

y_pred_train_1 = pd.DataFrame(y_pred_train_1, columns=["rf1_0", "rf1_1"], index=X_train.index)
y_pred_test_1 = pd.DataFrame(y_pred_test_1, columns=["rf1_0", "rf1_1"], index=X_test.index)

In [16]:
rf2.fit(X_train, y_train)
y_pred_train_2 = rf2.predict_proba(X_train)
y_pred_test_2 = rf2.predict_proba(X_test)

y_pred_train_2 = pd.DataFrame(y_pred_train_2, columns=["rf2_0", "rf2_1"], index=X_train.index)
y_pred_test_2 = pd.DataFrame(y_pred_test_2, columns=["rf2_0", "rf2_1"], index=X_test.index)

In [17]:
hidden_train_1 = pd.concat([X_train, y_pred_train_1, y_pred_train_2], axis=1)
hidden_test_1 = pd.concat([X_test, y_pred_test_1, y_pred_test_2], axis=1)

In [18]:
hidden_train_1.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,rf1_0,rf1_1,rf2_0,rf2_1
493,1,71.0,0,0,49.5042,0,1,1,0,0,0.626112,0.373888,0.755285,0.244715
779,1,43.0,0,1,211.3375,1,0,0,0,1,0.097917,0.902083,0.000000,1.000000
153,3,40.5,0,2,14.5000,0,1,0,0,1,0.802091,0.197909,0.887265,0.112735
621,1,42.0,1,0,52.5542,0,1,0,0,1,0.650493,0.349507,0.435268,0.564732
234,2,24.0,0,0,10.5000,0,1,0,0,1,0.889424,0.110576,0.972478,0.027522


In [19]:
rf3 = RandomForestClassifier(n_estimators=300, n_jobs=-1)
rf3.fit(hidden_train_1, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [20]:
y_pred3 = rf3.predict_proba(hidden_test_1)

In [21]:
roc_auc_score(y_test, y_pred3[:, 1])

0.84199999999999997

## With API

In [22]:
from deepforest.layer import Layer, InputLayer

In [23]:
input_layer = InputLayer(RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4),
                         RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10))

In [24]:
hidden_layer = Layer(input_layer,
                     RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=4),
                     RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10))

In [25]:
hidden_layer.fit(X_train, y_train)

In [26]:
hidden_layer.predict(X_test)

,model0_0,model0_1,model1_0,model1_1
741,0.904868,0.095132,1.000000,0.000000
224,0.904868,0.095132,1.000000,0.000000
313,0.981422,0.018578,0.997500,0.002500
254,0.015833,0.984167,0.012857,0.987143
188,0.974776,0.025224,0.977500,0.022500
154,0.990357,0.009643,0.999545,0.000455
140,0.015788,0.984212,0.000000,1.000000
316,0.001596,0.998404,0.000000,1.000000
552,0.969267,0.030733,0.952589,0.047411
60,0.163292,0.836708,0.032000,0.968000


# On teste plus loin

In [66]:
def random_forest_generator():
    for i in range(2, 15, 2):
        yield RandomForestClassifier(n_estimators=100,
                                     n_jobs=-1,
                                     min_samples_leaf=5,
                                     max_depth=i)
    for i in range(2, 15, 2):
        yield RandomForestClassifier(n_estimators=100,
                                     n_jobs=-1,
                                     max_features=1,
                                     min_samples_leaf=5,
                                     max_depth=i)

In [67]:
def input_layer():
    return InputLayer(*random_forest_generator())

In [68]:
def hidden_layer(layer):
    return Layer(layer, *random_forest_generator())

In [69]:
def output_layer(layer):
    return Layer(layer,
                 RandomForestClassifier(n_estimators=500,
                                        n_jobs=-1,
                                        min_samples_leaf=5,
                                        max_depth=10))

In [70]:
input_l = input_layer()
hidden_1 = hidden_layer(input_l)
hidden_2 = hidden_layer(hidden_1)
hidden_3 = hidden_layer(hidden_2)
hidden_4 = hidden_layer(hidden_3)
output_l = output_layer(hidden_4)

In [74]:
output_l.fit(X_train, y_train)

In [75]:
y_pred = output_l.predict(X_test)

In [76]:
roc_auc_score(y_test, y_pred.iloc[:, 1])

0.81224489795918364